In [17]:
from student import *
from torchsummary import summary
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import os
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report
import onnxruntime as ort
import os
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm import tqdm

In [ ]:
def dataset_creator(path_to_folder, max_by_experiment = 3):
    col, fire, steer = 0, 0, 0
    images, labels = [], []

    for folder in os.listdir(path_to_folder):

        if os.path.isdir(os.path.join(path_to_folder, folder)):

            for file in os.listdir(os.path.join(path_to_folder, folder)):

                if file.endswith('.txt'):
                    if 'labels' in file and col < max_by_experiment:
                        col += 1
                        for pic in sorted(os.listdir(os.path.join(path_to_folder, folder, 'images'))):
                            img = load_img(os.path.join(os.path.join(path_to_folder, folder, 'images'), pic), target_size=(200, 200), color_mode='grayscale')  
                            img_array = img_to_array(img) / 128.0 -1 
                            images.append(img_array)
                        
                        labels_txt = np.loadtxt(os.path.join(path_to_folder, folder, file))

                        for label in labels_txt:
                            if label == 0:
                                label = [np.array([1, 0]), np.array([np.nan]*4), np.array([np.nan])]
                            elif label == 1:
                                label = [np.array([0, 1]), np.array([np.nan]*4), np.array([np.nan])]
                            label = pad_sequences(label, dtype='float32', padding='post', value=np.nan)
                            labels.append(label)


                    elif 'fire' in file and fire < max_by_experiment:
                        fire += 1
                        for pic in sorted(os.listdir(os.path.join(path_to_folder, folder, 'images'))):
                            img = load_img(os.path.join(os.path.join(path_to_folder, folder, 'images'), pic), target_size=(200, 200), color_mode='grayscale')  
                            img_array = img_to_array(img) / 128.0 -1  
                            images.append(img_array)
                        
                        labels_txt = np.loadtxt(os.path.join(path_to_folder, folder, file), delimiter=' ')

                        for label in labels_txt:
                            label = [np.array([np.nan]*2), label, np.array([np.nan])]
                            label = pad_sequences(label, dtype='float32', padding='post', value=np.nan)
                            labels.append(label)

                            
                    elif 'sync' in file and steer < max_by_experiment:
                        steer += 1
                        for pic in sorted(os.listdir(os.path.join(path_to_folder, folder, 'images'))):
                            img = load_img(os.path.join(os.path.join(path_to_folder, folder, 'images'), pic), target_size=(200, 200), color_mode='grayscale')
                            img_array = img_to_array(img) / 128.0 - 1
                            images.append(img_array)

                        labels_txt = np.loadtxt(os.path.join(path_to_folder, folder, file), usecols=0, delimiter=',', skiprows=1)

                        for label in labels_txt:
                            label = [np.array([np.nan]*2), np.array([np.nan]*4), np.array([label])]
                            label = pad_sequences(label, dtype='float32', padding='post', value=np.nan)
                            labels.append(label)

    return np.array(images), np.array(labels)

images_test, labels_test = dataset_creator('../../../testing', 5)
indices = np.random.permutation(images_test.shape[0])
images_test = images_test[indices]
images_test = torch.tensor(images_test).permute(0, 3, 1, 2)
labels_test = labels_test[indices]
y_col_test, y_fire_test, y_steer_test = labels_test[:,0, :][:, :2], labels_test[:, 1, :], labels_test[:, 2, :][:, 0]
y_col_test, y_fire_test, y_steer_test = torch.tensor(y_col_test), torch.tensor(y_fire_test), torch.tensor(y_steer_test)

In [19]:
model = StudentOneEE()
state = torch.load('trained_model.pth')
model.load_state_dict(state)

model.eval()

/tmp/ipykernel_11080/3220605188.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load('trained_model.pth')


StudentOneEE(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilati

In [20]:
summary(model, (1, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 200, 200]             160
       BatchNorm2d-2         [-1, 16, 200, 200]              32
         MaxPool2d-3         [-1, 16, 100, 100]               0
            Conv2d-4         [-1, 32, 100, 100]           4,640
       BatchNorm2d-5         [-1, 32, 100, 100]              64
         MaxPool2d-6           [-1, 32, 50, 50]               0
            Linear-7                    [-1, 4]         320,004
            Conv2d-8           [-1, 64, 50, 50]          18,496
       BatchNorm2d-9           [-1, 64, 50, 50]             128
        MaxPool2d-10           [-1, 64, 25, 25]               0
           Conv2d-11          [-1, 128, 25, 25]          73,856
      BatchNorm2d-12          [-1, 128, 25, 25]             256
        MaxPool2d-13          [-1, 128, 12, 12]               0
        MaxPool2d-14            [-1, 12

One milion parameters compared to the 160 million used before.

### Size comparison

In [21]:
model_original = os.path.getsize('../../early_exit/ptq_and_ee_implementation/trained_model.onnx')/(1024**2)
model_kd = os.path.getsize('trained_model.onnx')/(1024**2)
model_kd_ptq = os.path.getsize('quantized_model.onnx')/(1024**2)

In [22]:
print(f'Original model size: {model_original} MB')
print(f'Knowledge distillation model size: {model_kd} MB')
print(f'Knowledge distillation model with PTQ size: {model_kd_ptq} MB')

Original model size: 650.882758140564 MB
Knowledge distillation model size: 3.9164791107177734 MB
Knowledge distillation model with PTQ size: 0.9964971542358398 MB


### Performance metrics Knowledge distillation

In [ ]:
def evaluar_modelo_onnx(modelo_onnx_path, entradas, labels_cls1, labels_cls2, labels_bin, labels_reg):
    """
    Función para evaluar la performance de un modelo ONNX en tareas de clasificación y regresión.

    Args:
        modelo_onnx_path (str): Ruta al archivo ONNX del modelo.
        entradas (list): Lista de entradas para el modelo.
        labels_cls1 (list): Etiquetas de clasificación multiclase 1.
        labels_cls2 (list): Etiquetas de clasificación multiclase 2.
        labels_bin (list): Etiquetas de clasificación binaria.
        labels_reg (list): Etiquetas de regresión.

    Returns:
        dict: Diccionario con las métricas de evaluación.
    """
    # onnx runtime
    session = ort.InferenceSession(modelo_onnx_path)

    all_pred_cls1 = []
    all_pred_cls2 = []
    all_pred_bin = []
    all_pred_reg = []

    all_labels_cls1 = []
    all_labels_cls2 = []
    all_labels_bin = []
    all_labels_reg = []

    for i in tqdm(range(len(entradas))):
        input_data = entradas[i].numpy().astype(np.float32)  
        label_cls1 = labels_cls1[i]
        label_cls2 = labels_cls2[i]
        label_bin = labels_bin[i]
        label_reg = labels_reg[i]

        outputs = session.run(None, {session.get_inputs()[0].name: input_data[np.newaxis, ...]})
        logits_cls1, logits_cls2, logits_bin, logits_reg = outputs

        if not np.isnan(label_cls1).any():
            pred_cls1 = np.argmax(logits_cls1, axis=1)[0]  
            all_pred_cls1.append(pred_cls1)
            all_labels_cls1.append(np.argmax(label_cls1))  

        if not np.isnan(label_cls2).any():
            pred_cls2 = np.argmax(logits_cls2, axis=1)[0]  
            all_pred_cls2.append(pred_cls2)
            all_labels_cls2.append(np.argmax(label_cls2))  

        if not np.isnan(label_bin).any():
            pred_bin = (logits_bin[0][0] > 0.5).astype(int)  
            all_pred_bin.append(pred_bin)
            all_labels_bin.append(int(label_bin))

        if not np.isnan(label_reg).any():
            pred_reg = logits_reg[0][0]  
            all_pred_reg.append(pred_reg)
            all_labels_reg.append(label_reg)

    acc_cls1 = accuracy_score(all_labels_cls1, all_pred_cls1) if all_labels_cls1 else None
    acc_cls2 = accuracy_score(all_labels_cls2, all_pred_cls2) if all_labels_cls2 else None
    acc_bin = accuracy_score(all_labels_bin, all_pred_bin) if all_labels_bin else None
    mse_reg = mean_squared_error(all_labels_reg, all_pred_reg) if all_labels_reg else None
    
    print(f'Precisión clasificación multiclase 1: {acc_cls1:.4f}' if acc_cls1 is not None else 'No hay datos para cls1')
    print(f'Precisión clasificación multiclase 2: {acc_cls2:.4f}' if acc_cls2 is not None else 'No hay datos para cls2')
    print(f'Precisión clasificación binaria: {acc_bin:.4f}' if acc_bin is not None else 'No hay datos para binaria')
    print(f'Error cuadrático medio (MSE) en regresión: {mse_reg:.4f}' if mse_reg is not None else 'No hay datos para regresión')

    return {
        'acc_cls1': acc_cls1,
        'acc_cls2': acc_cls2,
        'acc_bin': acc_bin,
        'mse_reg': mse_reg
    }

In [24]:
evaluar_modelo_onnx('trained_model.onnx', images_test, y_col_test, y_fire_test, y_steer_test, y_steer_test)

  0%|          | 0/5157 [00:00<?, ?it/s]/tmp/ipykernel_11080/890785527.py:43: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  if not np.isnan(label_cls1).any():
/tmp/ipykernel_11080/890785527.py:48: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  if not np.isnan(label_cls2).any():
/tmp/ipykernel_11080/890785527.py:53: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  if not np.isnan(label_bin).any():
/tmp/ipykernel_11080/890785527.py:58: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  if not np.isnan(label_reg).any():
100%|██████████| 5157/5157 [00:13<00:00, 369.75it/s]

Precisión clasificación multiclase 1: 0.1977
Precisión clasificación multiclase 2: 0.8112
Precisión clasificación binaria: 0.8455
Error cuadrático medio (MSE) en regresión: 0.0145


{'acc_cls1': 0.19774718397997496,
 'acc_cls2': 0.8112416107382551,
 'acc_bin': 0.8454913880445796,
 'mse_reg': np.float32(0.014537488)}

To compare with the results got in /home/esteban/Desktop/opt_dnn_embedded/three_tasks_one_ee_model/early_exit/results/results.ipynb for the original version.

### Performance metrics KD + PTQ

In [25]:
evaluar_modelo_onnx('quantized_model.onnx', images_test, y_col_test, y_fire_test, y_steer_test, y_steer_test)

  0%|          | 0/5157 [00:00<?, ?it/s]/tmp/ipykernel_11080/890785527.py:43: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  if not np.isnan(label_cls1).any():
/tmp/ipykernel_11080/890785527.py:48: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  if not np.isnan(label_cls2).any():
/tmp/ipykernel_11080/890785527.py:53: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  if not np.isnan(label_bin).any():
/tmp/ipykernel_11080/890785527.py:58: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  if not np.isnan(label_reg).any():
100%|██████████| 5157/5157 [00:24<00:00, 206.88it/s]


Precisión clasificación multiclase 1: 0.2065
Precisión clasificación multiclase 2: 0.8154
Precisión clasificación binaria: 0.8374
Error cuadrático medio (MSE) en regresión: 0.0155


{'acc_cls1': 0.2065081351689612,
 'acc_cls2': 0.8154362416107382,
 'acc_bin': 0.837386018237082,
 'mse_reg': np.float32(0.015500564)}